In [1]:
# #bp神经网络

#数据提取和处理
import re
def encode_label(label_list: list) -> (list, list):
    label_name = [[] for i in range(5)]

    label_encoded_list = []

    for label in label_list:
        if label is None:
            label_encoded_list.append(None)
            continue
        label_encoded = []

        for i in range(len(label) - 1):
            if not label[i] in label_name[i]:
                label_name[i].append(label[i])
                label_encoded.append(len(label_name[i]) - 1)
            else:
                label_encoded.append(label_name[i].index(label[i]))

        if label[4] is None:
            label_encoded.append([None])
        else:
            prop = []
            for ll in label[4]:
                if not ll in label_name[4]:
                    label_name[4].append(ll)
                    prop.append(len(label_name[4]) - 1)
                else:
                    prop.append(label_name[4].index(ll))
            label_encoded.append(prop)

        label_encoded_list.append(label_encoded)
    return label_name, label_encoded_list


def read_label(path: str) -> (list, list):
    label_list = []
    img_name = []
    with open(path, "r") as f:
        r = f.readline()
        while r:
            img_name.append(r[:5].strip())
            t = re.search('_missing descriptor', r)
            if t is not None:
                label_list.append(None)
                r = f.readline()
                continue
            else:
                label1 = []
                pattern = re.compile(r'\((.*?)\)')
                t = pattern.findall(r)
                for s in t[0:-1]:
                    c = s.split(" ")[-1].strip()
                    label1.append(c)

                pattern = re.compile(r'\'\((.*?)\)')
                t = pattern.findall(r)[0].strip()
                if not t == '':
                    prop = []
                    t = t.split(" ")
                    for i in t:
                        prop.append(i)
                    label1.append(prop)
                else:
                    label1.append(None)
            label_list.append(label1)
            r = f.readline()
    return label_list, img_name


def one_hot_prop(label_encoded_list: list, prop_num: int) -> list:
    label_list = []
    iii = 0
    for label in label_encoded_list:
        iii += 1
        if label is not None:
            prop_one_hot = [0 for i in range(prop_num)]
            prop = label.pop()
            for i in prop:
                if i is not None:
                    prop_one_hot[i] = 1
            label.append(prop_one_hot)
            label_list.append(label)
            if 1 in prop_one_hot:
                pass
        else:
            label_list.append(None)
    return label_list


def one_hot(index, num):
    label_one_hot = [0 for i in range(num)]
    label_one_hot[index] = 1
    return label_one_hot


def one_hot_label(label_list: list, label_name) -> list:
    label_one_hot = []
    for label in label_list:
        if label is not None:
            l = []
            for i in range(len(label_name) - 1):
                o = one_hot(label[i], len(label_name[i]))
                l += o
            l += label[-1]
        else:
            label_one_hot.append(None)
            continue
        label_one_hot.append(l)
    return label_one_hot


def label_pre_one_hot(path1, path2):
    label_list1, img_name1 = read_label(path1)
    # print(len(img_name1))
    label_list2, img_name2 = read_label(path2)
    # print(len(img_name2))
    label_list = label_list1 + label_list2

    male = 0
    total = 0

    # for label in label_list:
    #     if label is not None:
    #         total += 1
    #         if label[0] == 'male':
    #             male += 1
    # print(male, total)

    img_name = img_name1 + img_name2
    label_name, label_encoded_list = encode_label(label_list)
    prop_num = len(label_name[-1])
    label_list = one_hot_prop(label_encoded_list, prop_num)
    label_one_hot = one_hot_label(label_list, label_name)

    return label_one_hot, img_name, label_name


In [2]:
import raw_data_read_test as rd
import label_pre as lp
import cv2
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn
from sklearn import metrics
from sklearn import tree
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

# 训练集
channels = 1  # 图像的通道数，灰度图为1
path_rawdata = r"./rawdata"
path_label1 = r"./faceDR"
path_label2 = r"./faceDS"
label_one_hot,img_name,label_list = label_pre_one_hot(path_label1,path_label2)
img_list = rd.read_rawdata(channels, path_rawdata, img_name)  # 图片矩阵

n = 0
for i in range(len(label_list)):
    num = i - n
    if label_list[num] is None:
        del label_list[num]
        del img_name[num]
        n = n + 1

array_of_img = []
for img in img_list:
    if img is not None:
        img = img / 255.0
        img = cv2.resize(img, (100, 100))
        array_of_img.append(img)
train_images = np.array(array_of_img)
array_of_img = []

#卷积标签数值化，男为0，女为1
array_of_labels = []
for label in label_one_hot:
    if label is not None:
        append_label = label[0]
        array_of_labels.append(int(append_label))
train_labels = np.array(array_of_labels)
array_of_labels = []

# 划分数据集和训练集
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2, random_state=3)

#更改图像维度，方便训练
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# 特征降维(进行特征降维能大大减少训练时间但会一定程度的下降训练准确率)
pca = PCA(n_components=85)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

#归一化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# solver='lbfgs',  MLP的求解方法：L-BFGS 在小数据上表现较好，Adam 较为鲁棒，SGD在参数调整较优时会有最佳表现（分类效果与迭代次数）；SGD标识随机梯度下降。
# alpha:L2的参数：MLP是可以支持正则化的，默认为L2，具体参数需要调整
# hidden_layer_sizes=(5, 2) hidden层2层,第一层5个神经元，第二层2个神经元)，2层隐藏层，也就有3层神经网络
bp = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5,2), random_state=1) 
bp.fit(X_train_std, y_train)

print('每层网络层系数矩阵维度：\n',[coef.shape for coef in bp.coefs_])

# y_pred_pro =bp.predict_proba(X_test_std)
# print('预测结果概率：\n',y_pred_pro)

score = bp.score(X_test_std,y_test)
print('bp神经网络训练准确率：',score*100,'%')


数据缺失： 1228
数据缺失： 1232
数据缺失： 1808
数据缺失： 4056
数据缺失： 4135
数据缺失： 4136
数据缺失： 5004
每层网络层系数矩阵维度：
 [(85, 5), (5, 2), (2, 1)]
bp神经网络训练准确率： 85.80246913580247 %


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [8]:
bp.fit(X_train_std, y_train)
score = bp.score(X_test_std,y_test)
print('bp神经网络训练准确率：',score*100,'%')

bp神经网络训练准确率： 86.41975308641975 %


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
import numpy as np

k_fold = KFold(n_splits=10)
bp = MLPClassifier(random_state=1) 
params = {"hidden_layer_sizes": [(100,), (100,30),(5,2)],
              "solver": ['adam', 'sgd', 'lbfgs'],
              "alpha" : np.arange(1e-5,1e-4,1e-5),
              "max_iter": [20],
              "verbose": [True]
                             }
scoring_fnc = make_scorer(accuracy_score)
grid = GridSearchCV(bp, param_grid=params,scoring=scoring_fnc,cv=k_fold)
grid = grid.fit(X_train, y_train)

Iteration 1, loss = 1.20978957
Iteration 2, loss = 0.67519885
Iteration 3, loss = 0.53603537
Iteration 4, loss = 0.47358087
Iteration 5, loss = 0.43297720
Iteration 6, loss = 0.40458577
Iteration 7, loss = 0.38200669
Iteration 8, loss = 0.36358218
Iteration 9, loss = 0.34768485
Iteration 10, loss = 0.33381486
Iteration 11, loss = 0.32160122
Iteration 12, loss = 0.31029795
Iteration 13, loss = 0.29991628
Iteration 14, loss = 0.28968320
Iteration 15, loss = 0.28155804
Iteration 16, loss = 0.27300290
Iteration 17, loss = 0.26462829
Iteration 18, loss = 0.25787625
Iteration 19, loss = 0.25048780
Iteration 20, loss = 0.24322524
Iteration 1, loss = 1.21715620
Iteration 2, loss = 0.67760223


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 3, loss = 0.53749432
Iteration 4, loss = 0.47409485
Iteration 5, loss = 0.43366071
Iteration 6, loss = 0.40533201
Iteration 7, loss = 0.38247036
Iteration 8, loss = 0.36376137
Iteration 9, loss = 0.34791250
Iteration 10, loss = 0.33399868
Iteration 11, loss = 0.32174217
Iteration 12, loss = 0.31038155
Iteration 13, loss = 0.30023547
Iteration 14, loss = 0.28987876
Iteration 15, loss = 0.28156661
Iteration 16, loss = 0.27335400
Iteration 17, loss = 0.26507884
Iteration 18, loss = 0.25764856
Iteration 19, loss = 0.25038362
Iteration 20, loss = 0.24293357
Iteration 1, loss = 1.21515083
Iteration 2, loss = 0.67776501


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 3, loss = 0.53879720
Iteration 4, loss = 0.47673461
Iteration 5, loss = 0.43673329
Iteration 6, loss = 0.40838489
Iteration 7, loss = 0.38579809
Iteration 8, loss = 0.36718541
Iteration 9, loss = 0.35111677
Iteration 10, loss = 0.33678194
Iteration 11, loss = 0.32448262
Iteration 12, loss = 0.31320153
Iteration 13, loss = 0.30301219
Iteration 14, loss = 0.29241423
Iteration 15, loss = 0.28392093
Iteration 16, loss = 0.27559854
Iteration 17, loss = 0.26715162
Iteration 18, loss = 0.25947158
Iteration 19, loss = 0.25223420
Iteration 20, loss = 0.24496746
Iteration 1, loss = 1.19799529
Iteration 2, loss = 0.67089503


...

D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 7, loss = 0.76544410
Iteration 8, loss = 0.75864125
Iteration 9, loss = 0.75292137
Iteration 10, loss = 0.74783212
Iteration 11, loss = 0.74287414
Iteration 12, loss = 0.73839714
Iteration 13, loss = 0.73386853
Iteration 14, loss = 0.72950447
Iteration 15, loss = 0.72517572
Iteration 16, loss = 0.72109068
Iteration 17, loss = 0.71702573
Iteration 18, loss = 0.71294677
Iteration 19, loss = 0.70896311
Iteration 20, loss = 0.70521339


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_

Iteration 1, loss = 0.66570197
Iteration 2, loss = 0.51266041
Iteration 3, loss = 0.42829840
Iteration 4, loss = 0.36892586
Iteration 5, loss = 0.32782921
Iteration 6, loss = 0.30127020
Iteration 7, loss = 0.27571039
Iteration 8, loss = 0.25627350
Iteration 9, loss = 0.23838592
Iteration 10, loss = 0.22418565
Iteration 11, loss = 0.21041187
Iteration 12, loss = 0.19702465
Iteration 13, loss = 0.18123305
Iteration 14, loss = 0.17000532
Iteration 15, loss = 0.15710438
Iteration 16, loss = 0.14631027
Iteration 17, loss = 0.13589449
Iteration 18, loss = 0.12765403
Iteration 19, loss = 0.11791335
Iteration 20, loss = 0.10885102


D:\anaconda\envs\tensorflow\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [40]:
clf = grid.best_estimator_
print ("k_fold Parameter 'hidden_layer_sizes' is {} for the optimal model.".format(clf.get_params()['hidden_layer_sizes']))
print ("k_fold Parameter 'solver' is {} for the optimal model.".format(clf.get_params()['solver']))
print ("k_fold Parameter 'alpha' is {} for the optimal model.".format(clf.get_params()['alpha']))

k_fold Parameter 'hidden_layer_sizes' is (100, 30) for the optimal model.
k_fold Parameter 'solver' is adam for the optimal model.
k_fold Parameter 'alpha' is 1e-05 for the optimal model.


In [42]:
score = grid.score(X_test,y_test)
print('最优参数的bp神经网络训练准确率：',score*100,'%')

最优参数的bp神经网络训练准确率： 88.88888888888889 %


In [6]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
bp = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(100,3), random_state=1) 
kfold = KFold(n_splits=10)
bp_scores = cross_val_score(bp, X_train, y_train, cv=kfold)
cv_cross_bp = cross_validate(bp, X_train, y_train, cv=kfold, scoring=('accuracy', 'f1'))
print("bp Accuracy: %0.2f (+/- %0.2f)" % (bp_scores.mean(), bp_scores.std() * 2))
print("bp测试结果：",cv_cross_bp)

bp Accuracy: 0.90 (+/- 0.03)
bp测试结果： {'fit_time': array([3.27562904, 2.89321375, 2.81044817, 2.59035492, 3.0734539 ,
       2.96186113, 2.82176208, 2.84555554, 3.94684196, 2.8883419 ]), 'score_time': array([0.00215697, 0.        , 0.008003  , 0.        , 0.        ,
       0.        , 0.00199318, 0.        , 0.00800252, 0.00800204]), 'test_accuracy': array([0.89717224, 0.90745501, 0.91002571, 0.88431877, 0.89974293,
       0.91002571, 0.86632391, 0.89203085, 0.90721649, 0.8943299 ]), 'test_f1': array([0.89949749, 0.9       , 0.90304709, 0.88607595, 0.8907563 ,
       0.90616622, 0.86666667, 0.9       , 0.9       , 0.89460154])}
